In [ ]:
import argparse
import os
import random
import time
from distutils.util import strtobool

import gym
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.distributions.normal import Normal
from torch.distributions.multivariate_normal import MultivariateNormal
from torch.utils.tensorboard import SummaryWriter

In [10]:
PERIOD = 7

# Env
import gym, json
from gym import spaces
from epipolicy.core.epidemic import construct_epidemic
from epipolicy.obj.act import construct_act

class EpiEnv(gym.Env):
    """Custom Environment that follows gym interface"""
    metadata = {'render.modes': ['human']}

    def __init__(self, session):
        super(EpiEnv, self).__init__()
        self.epi = construct_epidemic(session)
        total_population = np.sum(self.epi.static.default_state.obs.current_comp)
        obs_count = self.epi.static.compartment_count * self.epi.static.locale_count * self.epi.static.group_count
        action_count = 0
        action_param_count =  0
        for itv in self.epi.static.interventions:
            if not itv.is_cost:
                action_count += 1
                action_param_count += len(itv.cp_list)
        self.act_domain = np.zeros((action_param_count, 2), dtype=np.float64)
        index = 0
        for itv in self.epi.static.interventions:
            if not itv.is_cost:
                for cp in itv.cp_list:
                    self.act_domain[index, 0] = cp.min_value
                    self.act_domain[index, 1] = cp.max_value
                    index += 1
        # Define action and observation space
        # They must be gym.spaces objects
        # Example when using discrete actions:
        self.action_space = spaces.Box(low=0, high=1, shape=(action_count,), dtype=np.float64)
        # Example for using image as input:
        self.observation_space = spaces.Box(low=0, high=total_population, shape=(obs_count,), dtype=np.float64)

    def step(self, action):
        expanded_action = np.zeros(len(self.act_domain), dtype=np.float64)
        index = 0
        for i in range(len(self.act_domain)):
            if self.act_domain[i, 0] == self.act_domain[i, 1]:
                expanded_action[i] = self.act_domain[i, 0]
            else:
                expanded_action[i] = action[index]
                index += 1

        epi_action = []
        index = 0
        for itv_id, itv in enumerate(self.epi.static.interventions):
            if not itv.is_cost:
                epi_action.append(construct_act(itv_id, expanded_action[index:index+len(itv.cp_list)]))
                index += len(itv.cp_list)

        total_r = 0
        for i in range(PERIOD):
            state, r, done = self.epi.step(epi_action)
            total_r += r
            if done:
                break
        return state.obs.current_comp.flatten(), total_r, done, dict()

    def reset(self):
        state = self.epi.reset()
        return state.obs.current_comp.flatten()  # reward, done, info can't be included
    def render(self, mode='human'):
        pass
    def close(self):
        pass

In [11]:
def parse_args(main_args = None):
    # fmt: off
    parser = argparse.ArgumentParser()
    parser.add_argument("--exp-name", type=str, default="PPO",
        help="the name of this experiment")
    parser.add_argument("--gym-id", type=str, default="HalfCheetahBulletEnv-v0",
        help="the id of the gym environment")
    parser.add_argument("--learning-rate", type=float, default=3e-4,
        help="the learning rate of the optimizer")
    parser.add_argument("--seed", type=int, default=1,
        help="seed of the experiment")
    parser.add_argument("--total-timesteps", type=int, default=700000,
        help="total timesteps of the experiments")
    parser.add_argument("--torch-deterministic", type=lambda x: bool(strtobool(x)), default=True, nargs="?", const=True,
        help="if toggled, `torch.backends.cudnn.deterministic=False`")
    parser.add_argument("--cuda", type=lambda x: bool(strtobool(x)), default=True, nargs="?", const=True,
        help="if toggled, cuda will be enabled by default")
    parser.add_argument("--track", type=lambda x: bool(strtobool(x)), default=False, nargs="?", const=True,
        help="if toggled, this experiment will be tracked with Weights and Biases")
    parser.add_argument("--wandb-project-name", type=str, default="ppo-implementation-details",
        help="the wandb's project name")
    parser.add_argument("--wandb-entity", type=str, default=None,
        help="the entity (team) of wandb's project")
    parser.add_argument("--capture-video", type=lambda x: bool(strtobool(x)), default=False, nargs="?", const=True,
        help="weather to capture videos of the agent performances (check out `videos` folder)")

    parser.add_argument("--policy_plot_interval", type=int, default=1,
        help="seed of the experiment")

    # Algorithm specific arguments
    parser.add_argument("--num-envs", type=int, default=1,
        help="the number of parallel game environments")
    parser.add_argument("--num-steps", type=int, default=2048,
        help="the number of steps to run in each environment per policy rollout")
    parser.add_argument("--anneal-lr", type=lambda x: bool(strtobool(x)), default=True, nargs="?", const=True,
        help="Toggle learning rate annealing for policy and value networks")
    parser.add_argument("--gae", type=lambda x: bool(strtobool(x)), default=True, nargs="?", const=True,
        help="Use GAE for advantage computation")
    parser.add_argument("--gamma", type=float, default=0.99,
        help="the discount factor gamma")
    parser.add_argument("--gae-lambda", type=float, default=0.95,
        help="the lambda for the general advantage estimation")
    parser.add_argument("--num-minibatches", type=int, default=32,
        help="the number of mini-batches")
    parser.add_argument("--update-epochs", type=int, default=10,
        help="the K epochs to update the policy")
    parser.add_argument("--norm-adv", type=lambda x: bool(strtobool(x)), default=True, nargs="?", const=True,
        help="Toggles advantages normalization")
    parser.add_argument("--clip-coef", type=float, default=0.2,
        help="the surrogate clipping coefficient")
    parser.add_argument("--clip-vloss", type=lambda x: bool(strtobool(x)), default=True, nargs="?", const=True,
        help="Toggles whether or not to use a clipped loss for the value function, as per the paper.")
    parser.add_argument("--ent-coef", type=float, default=0.0,
        help="coefficient of the entropy")
    parser.add_argument("--vf-coef", type=float, default=0.5,
        help="coefficient of the value function")
    parser.add_argument("--max-grad-norm", type=float, default=0.5,
        help="the maximum norm for the gradient clipping")
    parser.add_argument("--target-kl", type=float, default=None,
        help="the target KL divergence threshold")
    if main_args is not None:
        args = parser.parse_args(main_args.split())
    else:
        args = parser.parse_args()
    args.num_steps //= PERIOD
    args.total_timesteps //= PERIOD
    args.batch_size = int(args.num_envs * args.num_steps)
    args.minibatch_size = int(args.batch_size // args.num_minibatches)
    # fmt: on
    return args

In [12]:
epi_ids = ["SIR_A"]#, "SIR_B", "SIRV_A", "SIRV_B", "COVID_A", "COVID_B", "COVID_C"]

def make_env(gym_id, seed, idx, capture_video, run_name):
    def thunk():
        if gym_id in epi_ids:
            fp = open('jsons/{}.json'.format(gym_id), 'r')
            session = json.load(fp)
            env = EpiEnv(session)
        else:
            env = gym.make(gym_id)
        env = gym.wrappers.RecordEpisodeStatistics(env)
        if capture_video:
            if idx == 0:
                env = gym.wrappers.RecordVideo(env, f"videos/{run_name}")
        env = gym.wrappers.ClipAction(env)
        env = gym.wrappers.NormalizeObservation(env)
        env = gym.wrappers.TransformObservation(env, lambda obs: np.clip(obs, -10, 10))
        env = gym.wrappers.NormalizeReward(env)
        env = gym.wrappers.TransformReward(env, lambda reward: np.clip(reward, -10, 10))
        # Our env is deterministic
        # env.seed(seed)
        env.action_space.seed(seed)
        env.observation_space.seed(seed)
        return env

    return thunk

def make_primal_env(gym_id):
    def thunk():
        if gym_id in epi_ids:
            fp = open('jsons/{}.json'.format(gym_id), 'r')
            session = json.load(fp)
            env = EpiEnv(session)
        else:
            env = gym.make(gym_id)
        return env
    return thunk

In [157]:
def layer_init(layer, std=np.sqrt(2), bias_const=0.0):
    torch.nn.init.orthogonal_(layer.weight, std)
    torch.nn.init.constant_(layer.bias, bias_const)
    return layer


class Agent(nn.Module):
    def __init__(self, env):
        super(Agent, self).__init__()
        self.actor_mean_sigma = nn.Sequential(
            layer_init(nn.Linear(np.array(env.observation_space.shape).prod(), 64)),
            nn.Tanh(),
            layer_init(nn.Linear(64, 64)),
            nn.Tanh(),
            layer_init(nn.Linear(64, 2*np.prod(env.action_space.shape)), std=0.01),
        )
        self.m = MultivariateNormal(
            torch.zeros(np.prod(env.action_space.shape)), 
            torch.eye(np.prod(env.action_space.shape))
        )

    def get_action(self, x):
        actor_mean_sigma = self.actor_mean_sigma(x)
        action_mean = actor_mean_sigma[0,:np.prod(env.action_space.shape)]
        action_sigma = actor_mean_sigma[0,np.prod(env.action_space.shape):]
        epsilon = self.m.sample()
        
        action = action_mean + action_sigma * epsilon 
		
        # Apply the sigmoid to ensure the action is between 0 and 1
        action = torch.sigmoid(action)
        return action


In [158]:
env = make_primal_env(args.gym_id)()
agent = Agent(env)
optimizer = optim.Adam(agent.parameters(), lr=args.learning_rate, eps=1e-5)


In [ ]:
#### FROM SCRATCH IMPLEMENTATION OF SVG
NUM_SAMPLES = 10
UNROLL_HORIZON = 10
GAMMA = 0.99
NUM_UPDATES
# we need to store the rewards as we unroll

rewards = torch.zeros((UNROLL_HORIZON, NUM_SAMPLES), requires_grad=True)
actions = torch.zeros((UNROLL_HORIZON, NUM_SAMPLES) + env.action_space.shape)
obs = torch.zeros((UNROLL_HORIZON, NUM_SAMPLES) + env.observation_space.shape)

for update in range(0, NUM_UPDATES):
	for i in range(NUM_SAMPLES):
		next_obs = torch.Tensor(env.reset()).unsqueeze(0) # initial state
		for t in range(UNROLL_HORIZON):

			obs[t,i] = next_obs
			# ALGO LOGIC: action logic
			with torch.no_grad():
				action = agent.get_action(next_obs)
			actions[t,i] = action
			
			next_obs, reward, done, info = env.step(action.cpu().numpy())
			with torch.no_grad():
				rewards[t,i] = torch.tensor(reward, dtype=torch.float32)
			next_obs = torch.Tensor(next_obs).unsqueeze(0)

	# compute policy returns
	loss = torch.zeros((NUM_SAMPLES))
	for t in range(UNROLL_HORIZON):
		loss +=  GAMMA**t * rewards[t,:]

	mean_loss = loss.mean()

	optimizer.zero_grad()
	mean_loss.backward()
	optimizer.step()

In [160]:
obs

tensor([[[2.2244e+06, 1.0000e+02, 0.0000e+00],
         [2.2244e+06, 1.0000e+02, 0.0000e+00],
         [2.2244e+06, 1.0000e+02, 0.0000e+00],
         [2.2244e+06, 1.0000e+02, 0.0000e+00],
         [2.2244e+06, 1.0000e+02, 0.0000e+00],
         [2.2244e+06, 1.0000e+02, 0.0000e+00],
         [2.2244e+06, 1.0000e+02, 0.0000e+00],
         [2.2244e+06, 1.0000e+02, 0.0000e+00],
         [2.2244e+06, 1.0000e+02, 0.0000e+00],
         [2.2244e+06, 1.0000e+02, 0.0000e+00]],

        [[2.1923e+06, 1.2807e+02, 3.2132e+04],
         [2.1922e+06, 1.2801e+02, 3.2204e+04],
         [2.1922e+06, 1.2823e+02, 3.2242e+04],
         [2.1922e+06, 1.2809e+02, 3.2182e+04],
         [2.1922e+06, 1.2805e+02, 3.2165e+04],
         [2.1922e+06, 1.2808e+02, 3.2177e+04],
         [2.1922e+06, 1.2796e+02, 3.2218e+04],
         [2.1923e+06, 1.2792e+02, 3.2134e+04],
         [2.1922e+06, 1.2807e+02, 3.2165e+04],
         [2.1921e+06, 1.2785e+02, 3.2264e+04]],

        [[2.1601e+06, 1.5714e+02, 6.4297e+04],
         

In [161]:
actions

tensor([[[0.4974, 0.4975],
         [0.4985, 0.4982],
         [0.4991, 0.4957],
         [0.4982, 0.4973],
         [0.4979, 0.4978],
         [0.4981, 0.4975],
         [0.4987, 0.4988],
         [0.4974, 0.4992],
         [0.4979, 0.4976],
         [0.4994, 0.5000]],

        [[0.4975, 0.4987],
         [0.4984, 0.4949],
         [0.4986, 0.4952],
         [0.4987, 0.4957],
         [0.4984, 0.4980],
         [0.4993, 0.4960],
         [0.4973, 0.4964],
         [0.4980, 0.4981],
         [0.4983, 0.4991],
         [0.4979, 0.4948]],

        [[0.4978, 0.4970],
         [0.4989, 0.4994],
         [0.4984, 0.4961],
         [0.4973, 0.4961],
         [0.4982, 0.4970],
         [0.4982, 0.4952],
         [0.4982, 0.5014],
         [0.4986, 0.4975],
         [0.4970, 0.4972],
         [0.4990, 0.4966]],

        [[0.4980, 0.4973],
         [0.4977, 0.4973],
         [0.4992, 0.4982],
         [0.4980, 0.4970],
         [0.4981, 0.4973],
         [0.4983, 0.4967],
         [0.4986, 0.49

In [162]:
loss

tensor([-19045084., -19052208., -19063328., -19053060., -19048184., -19062802.,
        -19049478., -19046488., -19042106., -19055388.],
       grad_fn=<AddBackward0>)

In [163]:
mean_loss = loss.mean()
mean_loss

tensor(-19051812., grad_fn=<MeanBackward0>)

In [164]:
rewards

tensor([[-1805298.8750, -1808701.6250, -1808317.3750, -1807128.0000,
         -1806821.0000, -1807047.0000, -1809675.1250, -1806623.8750,
         -1806650.7500, -1812439.2500],
        [-1839678.2500, -1838968.5000, -1840207.1250, -1840628.0000,
         -1841508.3750, -1842238.8750, -1837413.3750, -1840244.6250,
         -1842057.8750, -1837561.5000],
        [-1876282.2500, -1881208.1250, -1877834.8750, -1874911.7500,
         -1877283.5000, -1876487.0000, -1880721.0000, -1878457.6250,
         -1874277.2500, -1879327.8750],
        [-1918479.6250, -1917720.0000, -1923142.5000, -1918874.8750,
         -1918662.5000, -1919500.1250, -1919224.8750, -1919669.7500,
         -1917587.2500, -1919044.2500],
        [-1965612.8750, -1966545.0000, -1966723.5000, -1963988.3750,
         -1965007.0000, -1966199.1250, -1963422.6250, -1962641.7500,
         -1964466.6250, -1965289.2500],
        [-2011988.3750, -2013915.6250, -2013621.0000, -2015078.3750,
         -2014179.3750, -2013984.2500, -2

In [ ]:
# from tqdm import tqdm
# args = parse_args("--gym-id SIR_A --seed 1")
# run_name = f"{args.gym_id}__{args.exp_name}__{args.seed}__{int(time.time())}"
# print("Running", run_name)
# writer = SummaryWriter(f"runs/{run_name}")
# writer.add_text(
#     "hyperparameters",
#     "|param|value|\n|-|-|\n%s" % ("\n".join([f"|{key}|{value}|" for key, value in vars(args).items()])),
# )

# # TRY NOT TO MODIFY: seeding
# random.seed(args.seed)
# np.random.seed(args.seed)
# torch.manual_seed(args.seed)
# torch.backends.cudnn.deterministic = args.torch_deterministic

# device = torch.device("cuda" if torch.cuda.is_available() and args.cuda else "cpu")

# # env setup
# envs = gym.vector.SyncVectorEnv(
#     [make_env(args.gym_id, args.seed + i, i, args.capture_video, run_name) for i in range(args.num_envs)]
# )
# # envs = gym.vector.AsyncVectorEnv(
# #     [make_env(args.gym_id, args.seed + i, i, args.capture_video, run_name) for i in range(args.num_envs)]
# # )
# test_env = make_primal_env(args.gym_id)()
# # test_env = make_env(args.gym_id, args.seed, 0, args.capture_video, run_name)()
# assert isinstance(envs.single_action_space, gym.spaces.Box), "only continuous action space is supported"

# agent = Agent(envs).to(device)
# optimizer = optim.Adam(agent.parameters(), lr=args.learning_rate, eps=1e-5)

# # ALGO Logic: Storage setup
# obs = torch.zeros((args.num_steps, args.num_envs) + envs.single_observation_space.shape).to(device)
# actions = torch.zeros((args.num_steps, args.num_envs) + envs.single_action_space.shape).to(device)
# logprobs = torch.zeros((args.num_steps, args.num_envs)).to(device)
# rewards = torch.zeros((args.num_steps, args.num_envs)).to(device)
# dones = torch.zeros((args.num_steps, args.num_envs)).to(device)
# values = torch.zeros((args.num_steps, args.num_envs)).to(device)

# # TRY NOT TO MODIFY: start the game
# global_step = 0
# start_time = time.time()
# next_obs = torch.Tensor(envs.reset()).to(device)
# next_done = torch.zeros(args.num_envs).to(device)
# num_updates = args.total_timesteps // args.batch_size
# ROLLOUT_HORIZON = 35

# csv_file = open('runs/{}/records.csv'.format(run_name), 'w')

# for update in tqdm(range(1, num_updates + 1)):
#     # Annealing the rate if instructed to do so.
#     if args.anneal_lr:
#         frac = 1.0 - (update - 1.0) / num_updates
#         lrnow = frac * args.learning_rate
#         optimizer.param_groups[0]["lr"] = lrnow

#     for step in range(0, args.num_steps):
#         global_step += 1 * args.num_envs
#         obs[step] = next_obs
#         dones[step] = next_done

#         # ALGO LOGIC: action logic
#         with torch.no_grad():
#             action, _, _, _ = agent.get_action(next_obs)
#         actions[step] = action

#         # TRY NOT TO MODIFY: execute the game and log data.
#         next_obs, reward, done, info = envs.step(action.cpu().numpy())
#         rewards[step] = torch.tensor(reward).to(device).view(-1)
#         next_obs, next_done = torch.Tensor(next_obs).to(device), torch.Tensor(done).to(device)

#         for item in info:
#             if "episode" in item.keys():
#                 print(f"global_step={global_step}, episodic_return={item['episode']['r']}")
#                 writer.add_scalar("charts/episodic_return", item["episode"]["r"], global_step)
#                 writer.add_scalar("charts/episodic_length", item["episode"]["l"], global_step)
#                 break

#     # bootstrap value if not done
#     with torch.no_grad():
#         returns = torch.zeros_like(rewards).to(device)
#         for t in reversed(range(args.num_steps)):
#             if t == args.num_steps - 1:
#                 nextnonterminal = 1.0 - next_done
#                 next_return = 0.0
#             else:
#                 nextnonterminal = 1.0 - dones[t + 1]
#                 next_return = returns[t + 1]
#             returns[t] = rewards[t] + args.gamma * nextnonterminal * next_return

#     # flatten the batch
#     b_obs = obs.reshape((-1,) + envs.single_observation_space.shape)
#     b_logprobs = logprobs.reshape(-1)
#     b_actions = actions.reshape((-1,) + envs.single_action_space.shape)
#     b_returns = returns.reshape(-1)

#     # Optimizing the policy and value network
#     b_inds = np.arange(args.batch_size)
#     clipfracs = []
#     for epoch in tqdm(range(max(1, args.update_epochs))): # eventually remove epoch as we look at trajectories only once
#         np.random.shuffle(b_inds)
#         for start in range(0, args.batch_size, args.minibatch_size):
#             # TODO: do we actually need mini-batches as we look at all trajectories at once?
#             end = start + args.minibatch_size
#             mb_inds = b_inds[start:end]

#             mb_returns = b_returns[mb_inds]
#             pg_loss = -mb_returns
            
#             loss = pg_loss 

#             optimizer.zero_grad()
#             loss.backward()
#             nn.utils.clip_grad_norm_(agent.parameters(), args.max_grad_norm)
#             optimizer.step()


#     y_pred, y_true = b_values.cpu().numpy(), b_returns.cpu().numpy()
#     var_y = np.var(y_true)
#     explained_var = np.nan if var_y == 0 else 1 - np.var(y_true - y_pred) / var_y

#     # TRY NOT TO MODIFY: record rewards for plotting purposes
#     writer.add_scalar("charts/learning_rate", optimizer.param_groups[0]["lr"], global_step)
#     writer.add_scalar("losses/value_loss", v_loss.item(), global_step)
#     writer.add_scalar("losses/policy_loss", pg_loss.item(), global_step)
#     writer.add_scalar("losses/entropy", entropy_loss.item(), global_step)
#     writer.add_scalar("losses/old_approx_kl", old_approx_kl.item(), global_step)
#     writer.add_scalar("losses/approx_kl", approx_kl.item(), global_step)
#     writer.add_scalar("losses/clipfrac", np.mean(clipfracs), global_step)
#     writer.add_scalar("losses/explained_variance", explained_var, global_step)
#     print("SPS:", int(global_step / (time.time() - start_time)))
#     writer.add_scalar("charts/SPS", int(global_step / (time.time() - start_time)), global_step)

#     # PLOT POLICY
#     if args.gym_id in epi_ids and (update - 1) % args.policy_plot_interval == 0:
#         test_obs = torch.Tensor(test_env.reset())
#         env_obs = torch.Tensor(envs.reset()).to(device)
#         timestep = 0
#         total_r = 0
#         done = False
#         itv_line = []
#         while not done:
#             with torch.no_grad():
#                 action_mean = agent.get_action_mean(env_obs)
#                 test_action_mean = torch.mean(action_mean, 0)
#                 test_action_mean = torch.clamp(test_action_mean, 0, 1)
             
#             test_obs, r, done, _ = test_env.step(test_action_mean.cpu().numpy())
#             test_obs = torch.Tensor(test_obs)
#             itv_index = 0
#             itv_array = []
#             for itv in test_env.epi.static.interventions:
#                 if not itv.is_cost:
#                     v = float(test_action_mean[itv_index])
#                     writer.add_scalar('charts/policy_{}/{}'.format(global_step, itv.name), v, timestep)
#                     itv_array.append(v)
#                     itv_index += 1
#             itv_line.append(itv_array)
            
#             env_obs, _, _, _ = envs.step(action_mean.cpu().numpy())
#             env_obs = torch.Tensor(env_obs).to(device)

#             total_r += r
#             timestep += PERIOD
            
#         line = '|'.join([str(global_step), str(total_r), str(itv_line)]) + '\n'
#         csv_file.write(line)
#         writer.add_scalar('charts/learning_curve', total_r, global_step)
#         print("At global step {}, total_rewards={}".format(global_step, total_r))

# csv_file.close()
# envs.close()
# writer.close()

Running SIR_A__PPO__1__1734145470


  0%|          | 0/342 [00:00<?, ?it/s]

global_step=53, episodic_return=-2089364992.0
global_step=106, episodic_return=-2093162112.0
global_step=159, episodic_return=-2092365824.0
global_step=212, episodic_return=-2094185600.0
global_step=265, episodic_return=-2092165888.0


  0%|          | 0/342 [00:29<?, ?it/s]


RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn